<a href="https://colab.research.google.com/github/Veerenderkumar/veerender_INFO5731_Fall2024/blob/main/kumar_veerender_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_amazon_reviews(url, num_reviews=1000):
    reviews = []
    page = 1
    while len(reviews) < num_reviews:
        current_url = f"{url}&pageNumber={page}"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
        }
        response = requests.get(current_url, headers=headers)
        if response.status_code != 200:
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', {'data-hook': 'review'})
        for review in review_elements:
            try:
                title = review.find('a', {'data-hook': 'review-title'})
                author = review.find('span', {'class': 'a-profile-name'})
                rating = review.find('i', {'data-hook': 'review-star-rating'})
                review_text = review.find('span', {'data-hook': 'review-body'})
                title_text = title.text.strip() if title else "No Title"
                author_text = author.text.strip() if author else "Anonymous"
                rating_text = rating.text.strip().split(" ")[0] if rating else "No Rating"
                review_body_text = review_text.text.strip() if review_text else "No Review"
                reviews.append({
                    'Title': title_text,
                    'Author': author_text,
                    'Rating': rating_text,
                    'Review': review_body_text
                })
            except Exception as e:
                pass
        if len(reviews) >= num_reviews:
            break
        page += 1
    return reviews

amazon_url = "https://www.amazon.com/dp/B06Y1YD5W7#customerReviews"
reviews_data = scrape_amazon_reviews(amazon_url, num_reviews=1000)
reviews_df = pd.DataFrame(reviews_data)
reviews_df.to_csv('amazon_cooker_reviews.csv', index=False)
print(reviews_df.head())
print(f"Collected {len(reviews_df)} reviews and saved to 'amazon_cooker_reviews.csv'")


                                               Title           Author Rating  \
0                  5.0 out of 5 stars\nLove the size  Amazon Customer    5.0   
1                    5.0 out of 5 stars\nWorks great         consumer    5.0   
2            5.0 out of 5 stars\nSmall yet Powerful!            David    5.0   
3                        5.0 out of 5 stars\nFor Dad    Dianna Tilley    5.0   
4  5.0 out of 5 stars\nWish I got the 6QT instead...              DLO    5.0   

                                              Review  
0  I wanted one of these to cook beans,  stews, r...  
1  My rice cooker stopped working.  Instead of pu...  
2  The Instant Pot Duo 7-in-1 Mini Electric Press...  
3  It arrived timely and in great condition.  It ...  
4  UPDATED - So I finally got the 6QT and oh. my....  
Collected 1004 reviews and saved to 'amazon_cooker_reviews.csv'


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [4]:
!pip install pandas nltk

import pandas as pd
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

file_path = 'amazon_cooker_reviews.csv'
df = pd.read_csv(file_path)

print(df.head())

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)
    text = text.lower()
    text = ' '.join(stemmer.stem(word) for word in text.split())
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return text

df['Cleaned_Review'] = df['Review'].apply(clean_text)

print(df[['Review', 'Cleaned_Review']].head())

df.to_csv('cleaned_amazon_cooker_reviews.csv', index=False)
print(f"Cleaned data saved to 'cleaned_amazon_cooker_reviews.csv'")





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                               Title           Author Rating  \
0                  5.0 out of 5 stars\nLove the size  Amazon Customer    5.0   
1                    5.0 out of 5 stars\nWorks great         consumer    5.0   
2            5.0 out of 5 stars\nSmall yet Powerful!            David    5.0   
3                        5.0 out of 5 stars\nFor Dad    Dianna Tilley    5.0   
4  5.0 out of 5 stars\nWish I got the 6QT instead...              DLO    5.0   

                                              Review  
0  I wanted one of these to cook beans,  stews, r...  
1  My rice cooker stopped working.  Instead of pu...  
2  The Instant Pot Duo 7-in-1 Mini Electric Press...  
3  It arrived timely and in great condition.  It ...  
4  UPDATED - So I finally got the 6QT and oh. my....  
                                              Review  \
0  I wanted one of these to cook beans,  stews, r...   
1  My rice cooker stopped working.  Instead of pu...   
2  The Instant Pot Du

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [6]:
!pip install pandas nltk spacy
!python -m spacy download en_core_web_sm

import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize, ne_chunk
from nltk.tree import Tree

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

df = pd.read_csv('cleaned_amazon_cooker_reviews.csv')

print(df.head())

nlp = spacy.load('en_core_web_sm')

def pos_analysis(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}
    for word, tag in tagged:
        if tag.startswith('N'):
            pos_counts['Noun'] += 1
        elif tag.startswith('V'):
            pos_counts['Verb'] += 1
        elif tag.startswith('J'):
            pos_counts['Adjective'] += 1
        elif tag.startswith('R'):
            pos_counts['Adverb'] += 1
    return pos_counts

df['POS_Counts'] = df['Cleaned_Review'].apply(pos_analysis)

print(df['POS_Counts'].iloc[0])

def parse_analysis(text):
    doc = nlp(text)
    constituency_tree = ne_chunk(pos_tag(word_tokenize(text)))
    dependency_tree = [(token.text, token.dep_, token.head.text) for token in doc]
    return constituency_tree, dependency_tree

constituency_tree, dependency_tree = parse_analysis(df['Cleaned_Review'].iloc[0])
print(constituency_tree)

for token in dependency_tree:
    print(token)

def named_entity_recognition(text):
    doc = nlp(text)
    entities = {'Person': 0, 'Organization': 0, 'Location': 0, 'Product': 0, 'Date': 0}
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            entities['Person'] += 1
        elif ent.label_ == 'ORG':
            entities['Organization'] += 1
        elif ent.label_ == 'GPE':
            entities['Location'] += 1
        elif ent.label_ == 'PRODUCT':
            entities['Product'] += 1
        elif ent.label_ == 'DATE':
            entities['Date'] += 1
    return entities

df['Named_Entities'] = df['Cleaned_Review'].apply(named_entity_recognition)

print(df['Named_Entities'].iloc[0])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


                                               Title           Author Rating  \
0                  5.0 out of 5 stars\nLove the size  Amazon Customer    5.0   
1                    5.0 out of 5 stars\nWorks great         consumer    5.0   
2            5.0 out of 5 stars\nSmall yet Powerful!            David    5.0   
3                        5.0 out of 5 stars\nFor Dad    Dianna Tilley    5.0   
4  5.0 out of 5 stars\nWish I got the 6QT instead...              DLO    5.0   

                                              Review  \
0  I wanted one of these to cook beans,  stews, r...   
1  My rice cooker stopped working.  Instead of pu...   
2  The Instant Pot Duo 7-in-1 Mini Electric Press...   
3  It arrived timely and in great condition.  It ...   
4  UPDATED - So I finally got the 6QT and oh. my....   

                                      Cleaned_Review  
0  want one cook bean stew rice etc u perfect muc...  
1  rice cooker stop work instead purchas anoth ri...  
2  instant pot du

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [7]:
from google.colab import files
files.download('cleaned_amazon_cooker_reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

The assignment was realistic but a bit not friendly as some sites didn't allow scraping. The assignment was thrilling
and helped me acquire NLP skills vital for working with text data. I look forward to such assignments although I
would love to see more time being allocated for a perfect work and better content mastery.
